# CLARISSA Code Examples

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rXPnEVc0du1su8Splil-8okX6kESwXI7)

**Click the badge above to run this notebook interactively in Google Colab!**

Executable Python code from the [Reservoir Basics Tutorial](https://irena-40cc50.gitlab.io/tutorials/reservoir-basics/).

---
## Setup (Team Members)

Run the cell below to connect to GitLab. Credentials are loaded automatically from the shared GDrive folder.

**First-time setup:** Add the shared [CLARISSA folder](https://drive.google.com/drive/folders/1qh0skTeyRNs4g9KwAhpd3J8Yj_XENIFs) to your GDrive (right-click > Organize > Add shortcut).

In [ ]:
#@title Mount GDrive & Clone Repository { display-mode: "form" }
#@markdown Credentials are loaded automatically from the shared CLARISSA folder.

import os
import json

# Mount Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print('Not in Colab - running locally')

# Load config from shared GDrive folder
CONFIG_PATHS = [
    '/content/drive/MyDrive/CLARISSA/clarissa_colab_config.json',  # Shortcut name
    '/content/drive/Shareddrives/CLARISSA/clarissa_colab_config.json',  # Shared Drive
    '/content/drive/MyDrive/clarissa_colab_config.json',  # Direct in MyDrive
]

config = None
GITLAB_TOKEN = None

if IN_COLAB:
    for config_path in CONFIG_PATHS:
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                config = json.load(f)
            GITLAB_TOKEN = config['gitlab']['token']
            print(f'Config loaded from: {config_path}')
            break
    
    if not config:
        print('Config not found! Add the CLARISSA shared folder to your GDrive:')
        print('https://drive.google.com/drive/folders/1qh0skTeyRNs4g9KwAhpd3J8Yj_XENIFs')
        print('(Right-click > Organize > Add shortcut to Drive)')

# Clone repo if token available
REPO_DIR = '/content/irena'

if GITLAB_TOKEN and not os.path.exists(REPO_DIR):
    repo_url = config['gitlab']['repo_url'].replace('https://', f'https://oauth2:{GITLAB_TOKEN}@')
    !git clone {repo_url} {REPO_DIR}
    print(f'Repository cloned to {REPO_DIR}')
elif os.path.exists(REPO_DIR):
    !cd {REPO_DIR} && git pull
    print('Repository updated')

if os.path.exists(REPO_DIR):
    os.chdir(REPO_DIR)
    print(f'Working directory: {os.getcwd()}')

---
## Pore Volume Calculation

In [ ]:
# Reservoir parameters (SPE9 model)
nx, ny, nz = 24, 25, 15  # Grid dimensions
dx, dy, dz = 300, 300, 50  # Cell sizes in feet
porosity = 0.087  # Average porosity
ntg = 1.0  # Net-to-gross ratio

# Calculate volumes
bulk_volume = nx * ny * nz * dx * dy * dz  # ft3
pore_volume = bulk_volume * porosity * ntg
pore_volume_bbl = pore_volume / 5.615  # Convert to barrels

print(f'Grid: {nx} x {ny} x {nz} = {nx*ny*nz:,} cells')
print(f'Pore volume: {pore_volume_bbl/1e6:.2f} MMbbl')

## Material Balance - OOIP

In [ ]:
def calculate_ooip(np_cum, bo, boi, delta_p, ce):
    """Calculate OOIP using simplified material balance."""
    return (np_cum * bo) / (boi * ce * delta_p)

# Example
N = calculate_ooip(
    np_cum=1_500_000,  # STB produced
    bo=1.25,
    boi=1.30,
    delta_p=500,  # psi
    ce=15e-6  # 1/psi
)

print(f'Estimated OOIP: {N/1e6:.1f} MMSTB')

## Recovery Factor Estimation

In [ ]:
# Typical recovery factors by drive mechanism
drive_mechanisms = {
    'Solution gas drive': (0.05, 0.30),
    'Gas cap drive': (0.20, 0.40),
    'Water drive': (0.35, 0.75),
    'Gravity drainage': (0.50, 0.70),
}

ooip_mmstb = 150  # Million STB

print(f'OOIP: {ooip_mmstb} MMSTB')
print()
print(f'{"Drive Mechanism":<25} {"RF Range":>15} {"Recoverable (MMSTB)":>20}')
print('-' * 62)

for mechanism, (rf_low, rf_high) in drive_mechanisms.items():
    rec_low = ooip_mmstb * rf_low
    rec_high = ooip_mmstb * rf_high
    print(f'{mechanism:<25} {rf_low*100:>5.0f}% - {rf_high*100:<5.0f}% {rec_low:>8.1f} - {rec_high:<8.1f}')

---
## Save Changes to GitLab

In [ ]:
#@title Commit & Push Changes { display-mode: "form" }
#@markdown Enter a commit message and run to push your changes.

commit_message = "docs(notebook): update from Colab"  #@param {type:"string"}

import os

if os.path.exists('/content/irena/.git'):
    os.chdir('/content/irena')
    !git config user.email "colab@clarissa.dev"
    !git config user.name "CLARISSA Colab"
    !git add -A
    !git status
    !git commit -m "{commit_message}" || echo "Nothing to commit"
    !git push
    print('Changes pushed to GitLab!')
else:
    print('Repository not cloned - run the setup cell first')